# SAE-97 - Nettoyage Dataset Reviews

**Objectif:** Charger, analyser et nettoyer le dataset Reviews

**Input:** `data/raw/yelp_academic_reviews4students.jsonl`

**Output:** `data/cleaned/reviews_clean.parquet`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
print("✅ Imports réussis")

In [ ]:
RAW_DATA_PATH = Path('../data/raw/yelp_academic_reviews4students.jsonl')
CLEANED_DATA_PATH = Path('../data/cleaned/reviews_clean.parquet')
CLEANED_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"📂 Chargement: {RAW_DATA_PATH}")
print(f"💾 Sauvegarde: {CLEANED_DATA_PATH}")

In [ ]:
print("⏳ Chargement... (1-2 min)")
reviews_df = pd.read_json(RAW_DATA_PATH, lines=True)
print(f"✅ {reviews_df.shape[0]:,} lignes, {reviews_df.shape[1]} colonnes")

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.info()

In [ ]:
missing = pd.DataFrame({
    'Colonne': reviews_df.columns,
    'Manquantes': reviews_df.isnull().sum(),
    'Pct': (reviews_df.isnull().sum() / len(reviews_df) * 100).round(2)
}).sort_values('Pct', ascending=False)
missing[missing['Manquantes'] > 0]

In [ ]:
reviews_clean = reviews_df.copy()
print(f"📋 Initial: {len(reviews_clean):,} lignes")

In [ ]:
# Supprimer doublons
dups = reviews_clean.duplicated(subset=['review_id']).sum()
if dups > 0:
    reviews_clean = reviews_clean.drop_duplicates(subset=['review_id'])
    print(f"❌ {dups} doublons supprimés")
else:
    print("✅ Aucun doublon")

In [ ]:
# Supprimer lignes sans données essentielles
for col in ['review_id', 'text', 'user_id', 'business_id']:
    if col in reviews_clean.columns:
        before = len(reviews_clean)
        reviews_clean = reviews_clean.dropna(subset=[col])
        if before > len(reviews_clean):
            print(f"❌ {before - len(reviews_clean)} sans {col}")

In [ ]:
# Nettoyer texte
if 'text' in reviews_clean.columns:
    reviews_clean['text_length'] = reviews_clean['text'].str.len()
    before = len(reviews_clean)
    reviews_clean = reviews_clean[reviews_clean['text_length'] >= 10]
    print(f"❌ {before - len(reviews_clean)} avis trop courts")
    
    reviews_clean['text'] = reviews_clean['text'].str.replace(r'\s+', ' ', regex=True).str.strip()
    print("✅ Texte nettoyé")

In [ ]:
# Valider notes
if 'stars' in reviews_clean.columns:
    before = len(reviews_clean)
    reviews_clean = reviews_clean[(reviews_clean['stars'] >= 1) & (reviews_clean['stars'] <= 5)]
    if before > len(reviews_clean):
        print(f"❌ {before - len(reviews_clean)} notes invalides")
    else:
        print("✅ Notes valides (1-5)")

In [ ]:
# Convertir dates
if 'date' in reviews_clean.columns:
    reviews_clean['date'] = pd.to_datetime(reviews_clean['date'], errors='coerce')
    invalid = reviews_clean['date'].isnull().sum()
    if invalid > 0:
        reviews_clean = reviews_clean.dropna(subset=['date'])
        print(f"❌ {invalid} dates invalides")
    else:
        print("✅ Dates converties")

In [ ]:
print("📊 RÉSUMÉ")
print("=" * 50)
print(f"Initial:    {len(reviews_df):,}")
print(f"Final:      {len(reviews_clean):,}")
print(f"Supprimé:   {len(reviews_df) - len(reviews_clean):,}")
print(f"Conservé:   {(len(reviews_clean) / len(reviews_df) * 100):.2f}%")

In [ ]:
# Distribution notes
if 'stars' in reviews_clean.columns:
    plt.figure(figsize=(10, 5))
    reviews_clean['stars'].value_counts().sort_index().plot(kind='bar')
    plt.xlabel('Note')
    plt.ylabel('Nombre')
    plt.title('Distribution des Notes')
    plt.show()

In [ ]:
# Sauvegarder
if 'text_length' in reviews_clean.columns:
    reviews_clean = reviews_clean.drop(columns=['text_length'])

print("⏳ Sauvegarde...")
reviews_clean.to_parquet(CLEANED_DATA_PATH, index=False, compression='snappy')
print(f"✅ Sauvegardé: {CLEANED_DATA_PATH}")

## ✅ SAE-97 Terminé